In [ ]:
import numpy as np
import json
import os
import shutil
import cv2
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
%matplotlib inline
from tqdm import tqdm

import sys
sys.path.append('/home/dan/work/cocoapi/PythonAPI/')

from pycocotools.coco import COCO

In [ ]:
DATA_DIR = '/home/gpu2/hdd/dan/COCO'
# DATA_TYPE = 'train2017'
DATA_TYPE = 'val2017'
# RESULT_PATH = '/home/gpu2/hdd/dan/COCO/train_annotations/'
RESULT_PATH = '/home/gpu2/hdd/dan/COCO/val_annotations/'

In [ ]:
coco = COCO('{}/annotations/instances_{}.json'.format(DATA_DIR, DATA_TYPE))

# Show categories

In [ ]:
categories = coco.loadCats(coco.getCatIds())
coco_id_to_name = {c['id']: c['name'] for c in categories}
names = [cat['name'] for cat in categories]
print('COCO categories: \n{}'.format(' '.join(names)))
print('\nnumber of labels:', len(names))

# Create label encoding

In [ ]:
# number of row - integer encoding
if not os.path.exists('coco_labels.txt'):
    with open('coco_labels.txt', 'w') as f:
        for n in names:
            f.write(n + '\n')

# Show an image with full annotation

In [ ]:
catIds = coco.getCatIds(catNms=['person', 'zebra'])
imgIds = coco.getImgIds(catIds=catIds)

In [ ]:
i = np.random.randint(0, len(imgIds))
metadata = coco.loadImgs(imgIds[i])[0]
image = cv2.imread('%s/images/%s/%s' % (DATA_DIR, DATA_TYPE, metadata['file_name']))
image = image[:, :, [2, 1, 0]]
annIds = coco.getAnnIds(imgIds=metadata['id'], catIds=catIds, iscrowd=False)
annotations = coco.loadAnns(annIds)

plt.imshow(image)
plt.axis('off')
coco.showAnns(annotations)

# Show bounding box annotation

In [ ]:
def draw_boxes_on_image(image, annotations):

    image = Image.fromarray(image)
    draw = ImageDraw.Draw(image, 'RGBA')
    width, height = image.size

    for a in annotations:
        xmin, ymin, w, h = a['bbox']
        xmax, ymax = xmin + w, ymin + h

        fill = (255, 255, 255, 45)
        outline = 'red'
        draw.rectangle(
            [(xmin, ymin), (xmax, ymax)],
            fill=fill, outline=outline
        )
    return image

In [ ]:
draw_boxes_on_image(image, annotations)

# Convert

In [ ]:
catIds = coco.getCatIds(catNms=['person', 'zebra'])
imgIds = coco.getImgIds(catIds=catIds)

imgIds = coco.getImgIds()
print('number of images:', len(imgIds))

In [ ]:
def get_annotation(i):
    metadata = coco.loadImgs(i)[0]
    annIds = coco.getAnnIds(imgIds=metadata['id'])
    height, width = metadata['height'], metadata['width']
    annotation = {
      "filename": metadata['file_name'],
      "size": {"depth": 3, "width": width, "height": height}
    }
    objects = []
    for a in coco.loadAnns(annIds):
        label = coco_id_to_name[a['category_id']]
        xmin, ymin, w, h = a['bbox']
        xmax, ymax = xmin + w, ymin + h
        
        ymin = min(ymin, ymax)
        xmin = min(xmin, xmax)
        ymax = max(ymin, ymax)
        xmax = max(xmin, xmax)
        
        ymin = min(max(0, ymin), height)
        xmin = min(max(0, xmin), width)
        ymax = max(min(height, ymax), 0)
        xmax = max(min(width, xmax), 0)
        
        if (ymax - ymin) < 1 or (xmax - xmin) < 1:
            print('sad!')
            continue

        objects.append({"bndbox": {"ymin": ymin, "ymax": ymax, "xmax": xmax, "xmin": xmin}, "name": label})

    annotation["object"] = objects
    return annotation

In [ ]:
shutil.rmtree(RESULT_PATH, ignore_errors=True)
os.mkdir(RESULT_PATH)

In [ ]:
for i in tqdm(imgIds):
    d = get_annotation(i)
    filename = d['filename']
    assert filename.endswith('.jpg')
    name = filename[:-4]
    json.dump(d, open(os.path.join(RESULT_PATH, name + '.json'), 'w')) 